## Neighborhoods Segmentation in Bangkok ##

This code is used to segment neighborhoods in Bangkok into clusters of similar characteristics. <br>
This capstone project is a part of the IBM data science certificate program on Coursera.

### Introduction/Business Problem ###

**Bangkok** is the capital city of Thailand and also the most crowd city in the country. From 2010 census conducted by the **National Statistic Office of Thailand** [1], the population in the city was over **8** millions. In 2019, it is estimated to home over **10** millions people making it 32th in the most populated city ranking, just a single place behind its neighbor **Ho Chi Minh City** in **Vietname** [2].

Even though the city is home to over **13%** of the country's total population, it is also an important international business areas as well as it privides facilities to more than 120,000 asian people and notably over 50,000 western people. Bangkok is considered a **primate city**, or one that serves as the population, political and financial center of a country with no other rival city[3].

It is clear that we can find great diversity in terms of businesses, people, and places in this city; however, are those diverse categories mingling together equally in all areas perfectly rendering the city into an ideal mixture or are there still several segments that we can tell apart? Do all areas in Bangkok play equal roles in term of businesses or living places? **Knowing that there are a lot of opportunities to be discovered in this well-populated place, to be able to clearly identifying where we should start some certain businesses will be a great idea and this is the problem that we are going to solve here.**

<img src="http://prod-upp-image-read.ft.com/9abfe4da-8342-11e7-94e2-c5b903247afd" title="Bangkok" />

References:<br>

[1]__[National Statistic Office of Thailand](http://web.nso.go.th/)__ <br>
[2]__[Capital City Population Ranking](http://www.citymayors.com/statistics/largest-cities-population-125.html)__<br>
[3]__[Bangkok as the Major City of Thailand](http://worldpopulationreview.com/world-cities/bangkok-population/)__

### Data ###

We will use data from two main sources in this analysis

1. **Location data** 

This will tell us about the name of each district area in Bangkok together with their Latitude and Longitude as Geographic coordinate that will be used to fetch the important places within each area from FourSquare API. In this study, the data will be scraped from geonames using the following site.__(https://www.geonames.org/postal-codes/TH/10/bangkok.html)__

2. **Popular places in each area** 

Since we will cluster areas in Bangkok based on popular places in each area, we will rely on FourSquare API "Explore" request.

### Part I: Obtain Neighborhoods in Bangkok ###
Step 1: Obtain raw data from wikipedia <br>
Step 2: Create DataFrame of Postal Code, Borough, and Neighborhood in Toronto <br>
Step 3: Clean and format the data<br>

In [25]:
#library installation section
import sys
!{sys.executable} -m pip install --upgrade pip
!{sys.executable} -m pip install beautifulsoup4
!{sys.executable} -m pip install geopy
!{sys.executable} -m pip install lxml
!{sys.executable} -m pip install geocoder
!{sys.executable} -m pip install folium

Requirement already up-to-date: pip in c:\users\chayutwo\anaconda3\lib\site-packages (19.1)


In [26]:
#import librabries
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import json
import requests
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium
import matplotlib.pyplot as plt
import re
print('all libraries are imported')

all libraries are imported


**Step 1: Obtain raw data from wikipedia** 

In [27]:
#Scrape Postal code, Borough, and Neighborhood name in Toronto from Wikipedia page
url = 'https://www.geonames.org/postal-codes/TH/10/bangkok.html'
result = requests.get(url).text
soup = BeautifulSoup(result, 'lxml')
print(soup)

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd ">
<html>
<head>
<title>Postal Codes Bangkok, Thailand</title>
<link href="https://www.geonames.org/opensearch-description.xml" rel="search" title="geonames" type="application/opensearchdescription+xml"/>
<link href="/geonames.ico" rel="shortcut icon"/>
<link href="/geonames.css" rel="StyleSheet" type="text/css"/>
</head>
<body>
<table cellpadding="0" cellspacing="0" id="topmenutable"><tr>
<td class="topmenu"> <a href="/" title="GeoName Home"> GeoNames Home</a> | <a href="/postal-codes/" title="Postal Codes">Postal Codes</a> | <a href="/export/" title="Database Dump and Webservice API">Download / Webservice</a> | <a href="/about.html" title="About GeoNames">About</a> </td>
<td class="topsearch">
<form action="/servlet/geonames" class="topsearch" method="get" name="searchForm">

      search <input class="topmenu" name="q" size="20" type="text" value=""/>
<input name="srv" type="hidden" 

**Step 2: Create DataFrame of Postal Code, Borough, and Neighborhood in Toronto**

In [28]:
def extractPostal(text):
    '''
    this function is used to extract the name of the Neighborhood, Postal code, and location of each district in Bangkok
    input: text containing string of district postal code and location ex. 4Pom Prap Sattru Phai10100ThailandBangkok   13.733/100.5
    output: tuple of Neighborhood name, Postal code, latitude, and longitude
    '''
    loc =re.search('\d{5}', text).span() #get the location of the postal code
    locstart = re.search('\D', text).span()[0]
    Neighborhood = text[locstart:loc[0]].strip()
    PostalCode = text[loc[0]: loc[1]].strip()
    return (PostalCode, Neighborhood)

def extractLocation(text):
    loc =re.search('/', text).span() #get the location of /
    Latitude = text[:loc[0]].strip()
    Longitude = text[loc[1]:].strip()
    return (float(Latitude), float(Longitude))

In [29]:
finalList = []
locationList = []
rows = soup.find_all('tr')
startBoo = False
for row in rows:
    cleantext = BeautifulSoup(str(row), 'lxml').get_text()
    if startBoo and cleantext.strip() != '':
        try:
            finalList.append(extractPostal(cleantext))
        except:
            locationList.append(extractLocation(cleantext))
    if 'PlaceCodeCountryAdmin1Admin2Admin3' ==  cleantext:
        #start of the data
        startBoo = True
#convert finalList to a DataFrame
df_postal = pd.DataFrame(finalList)
df_location = pd.DataFrame(locationList)
#rename the columns
df_postal.columns = ['PostalCode', 'Neighborhood']
df_location.columns = ['Latitude', 'Longitude']
df = pd.concat([df_postal,df_location], axis = 1)
df.loc[15,'Latitude'] = 13.877388 #correction for Lak Si
df.loc[15,'Longitude'] = 100.569273
df.loc[18,'Latitude'] = 13.819628 #correction for Khanna Yao
df.loc[18,'Longitude'] = 100.679240
df.loc[19,'Latitude'] = 13.825544 #correction for Lat Phrao
df.loc[19,'Longitude'] = 100.604447
print(df.shape)

(44, 4)


### Part III: Neighborhood Clustering ###
Step 1: Use FourSquare to explore each neighborhood <br>
Step 2: Use one hot encoding technique to obtain feature df for K means clustering <br>
Step 3: Clustering and making visualization<br>

**Step 1: Use FourSquare to explore each neighborhood**<br>
- Identify user credentials
- Use FourSquare API to send request url to obtain data regarding the popular venues in each neighborhood


In [30]:
# @hidden_cell
CLIENT_ID = 'QWVZSL1DJ0IJFLJCXBBPCKRLEPRZSV5B0IC1M2QDJ0TNFQSF' # your Foursquare ID
CLIENT_SECRET = 'DHHRSP5XUVGOZLNR3E0Z3NARGVJ1YSJVGXUB4FO52J34F13F' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: QWVZSL1DJ0IJFLJCXBBPCKRLEPRZSV5B0IC1M2QDJ0TNFQSF
CLIENT_SECRET:DHHRSP5XUVGOZLNR3E0Z3NARGVJ1YSJVGXUB4FO52J34F13F


In [31]:
def getNearbyVenues(names, latitudes, longitudes, radius=2000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [32]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [33]:
# type your answer here
LIMIT  =80
bangkok_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )



Wattana
Yannawa
Thawi Wattana
Pom Prap Sattru Phai
Samphanthawong
Bang Kholame
Thung Khru
Rat Burana
Bang Bon
Bang Khun Thian
Nong Khaem
Phasi Charoen
Bang Khae
Taling Chan
Phra Nakhon
Lak Si
Sai Mai
Bang Khen
Khanna Yao
Lat Phrao
Saphan Sung B
Bung Kum
Pra Vet
Suang Luang
Phra Khanong
Dusit
Wangthong Lang
Huai Khwang
Pahtum Wan
Ding Daeng
Bang Rak
Khlong Samwa
Min Buri
Lat Krabang
Nong Chok
Bangkok Yai
Khlong San
Thon Buri
Bang Plat
Bangkok Noi
Bang Su
Bang Kapi
Bang Na
Phaya Thai


**Step 2: Use one hot encoding technique to obtain feature df for K means clustering**<br>
- Use get_dummies function in pandas to get DataFrame containing dummy features of venue category in each neighborhood
- Group venues in the same neighborhood together to find the frequency of venues in each category

In [34]:
# one hot encoding
bangkok_onehot = pd.get_dummies(bangkok_venues['Venue Category'])

# add neighborhood column back to dataframe
bangkok_onehot['Neighborhood'] = bangkok_venues['Neighborhood']

# move neighborhood column to the first column
fixed_columns = [bangkok_onehot.columns[-1]] + list(bangkok_onehot.columns)[:-1]
bangkok_onehot = bangkok_onehot[fixed_columns]

bangkok_onehot.head(10)

,Yoga Studio,American Restaurant,Arcade,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,...,Train Station,Udon Restaurant,Vegetarian / Vegan Restaurant,Veterinarian,Vietnamese Restaurant,Warehouse Store,Water Park,Whisky Bar,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
bangkok_onehot.groupby('Neighborhood').sum().sum(axis = 1)

Neighborhood
Bang Bon                71
Bang Kapi               80
Bang Khae               80
Bang Khen               51
Bang Kholame            80
Bang Khun Thian         80
Bang Na                 80
Bang Plat               78
Bang Rak                80
Bang Su                 80
Bangkok Noi             78
Bangkok Yai             80
Bung Kum                80
Ding Daeng              80
Dusit                   80
Huai Khwang             80
Khanna Yao              80
Khlong Samwa            28
Khlong San              80
Lak Si                  80
Lat Krabang             80
Lat Phrao               80
Min Buri                71
Nong Chok               22
Nong Khaem              31
Pahtum Wan              80
Phasi Charoen           80
Phaya Thai              80
Phra Khanong            80
Phra Nakhon             80
Pom Prap Sattru Phai    80
Pra Vet                 51
Rat Burana              80
Sai Mai                 80
Samphanthawong          80
Saphan Sung B           80
Suang Luang    

In [36]:
#groupby neighborhood to get frequency of each venue in each neighborhood
bangkok_grouped = bangkok_onehot.groupby('Neighborhood').mean()
bangkok_grouped.reset_index(inplace = True)
bangkok_grouped.head()

,Neighborhood,Yoga Studio,American Restaurant,Arcade,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,...,Train Station,Udon Restaurant,Vegetarian / Vegan Restaurant,Veterinarian,Vietnamese Restaurant,Warehouse Store,Water Park,Whisky Bar,Wine Bar,Wings Joint
0,Bang Bon,0.0,0.0,0.014085,0.0,0.0,0.028169,0.0,0.0,0.0,...,0.028169,0.0,0.0,0.0,0.014085,0.0000,0.0000,0.0,0.0,0.0000
1,Bang Kapi,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.012500,0.0125,0.0125,0.0,0.0,0.0125
2,Bang Khae,0.0,0.0,0.000000,0.0,0.0,0.037500,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.0000,0.0125,0.0,0.0,0.0000
3,Bang Khen,0.0,0.0,0.000000,0.0,0.0,0.039216,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.0000,0.0000,0.0,0.0,0.0000
4,Bang Kholame,0.0,0.0,0.012500,0.0,0.0,0.012500,0.0,0.0,0.0,...,0.025000,0.0,0.0,0.0,0.012500,0.0000,0.0000,0.0,0.0,0.0000


To aid in exploring and creating summary from our information, we will rank top categories in each neighborhood

In [37]:
def return_most_common_venues(row, num):
    row_temp = row[1:].astype(float)
    row_temp_sorted = row_temp.sort_values(ascending = False)
    return row_temp_sorted.index.values[:num]

In [38]:
num = 10 #get only top 10 venue categories in each neighborhood
suffix = ['st', 'nd', 'rd']
columns_name = ['Neighborhood']
for i in range(num):
    try:
        columns_name.append('{}{} Most Common Venue'.format(i+1, suffix[i]))
    except:
        columns_name.append('{}th Most Common Venue'.format(i+1))
        
neighborhoods_venues_sorted = pd.DataFrame(columns = columns_name)
neighborhoods_venues_sorted['Neighborhood'] = bangkok_grouped['Neighborhood']
for i,row in bangkok_grouped.iterrows():
    neighborhoods_venues_sorted.iloc[i, 1:]=return_most_common_venues(row, num)
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bang Bon,Convenience Store,Noodle House,Coffee Shop,Thai Restaurant,Chinese Restaurant,Seafood Restaurant,Steakhouse,Supermarket,Ice Cream Shop,BBQ Joint
1,Bang Kapi,Coffee Shop,Thai Restaurant,Noodle House,Japanese Restaurant,Dessert Shop,Steakhouse,Som Tum Restaurant,Clothing Store,BBQ Joint,Electronics Store
2,Bang Khae,Noodle House,Coffee Shop,Thai Restaurant,Café,BBQ Joint,Dessert Shop,Japanese Restaurant,Ice Cream Shop,Hotpot Restaurant,Convenience Store
3,Bang Khen,Thai Restaurant,Noodle House,Coffee Shop,Convenience Store,Som Tum Restaurant,Seafood Restaurant,Asian Restaurant,Bakery,Steakhouse,Fast Food Restaurant
4,Bang Kholame,Convenience Store,Noodle House,Thai Restaurant,Coffee Shop,Som Tum Restaurant,Seafood Restaurant,Train Station,Gym / Fitness Center,Fast Food Restaurant,Market


**Step 3: Clustering and making visualization**<br>
- Using KMeans from sklearn.cluster to cluster neighborhoods into 4 groups
- Creat visualization using folium library
- Explore characteristics of each group

In [39]:
#import libraries for clustering
from sklearn.cluster import KMeans

# set number of clusters
kclusters =4

bangkok_grouped_clustering = bangkok_grouped.drop('Neighborhood', axis = 1)

# run k-means clustering
kmeans = KMeans(n_clusters = kclusters, random_state = 0).fit(bangkok_grouped_clustering)
# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 3, 3, 3, 0, 3, 3, 2, 2, 3])

In [40]:
neighborhoods_venues_sorted['Cluster Labels'] = kmeans.labels_

bangkok_final = df.merge(neighborhoods_venues_sorted, on = 'Neighborhood', how = 'inner')

bangkok_final.head()

,PostalCode,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
0,10110,Wattana,13.708,100.456,Convenience Store,Noodle House,Thai Restaurant,Coffee Shop,Som Tum Restaurant,Seafood Restaurant,Train Station,Gym / Fitness Center,Fast Food Restaurant,Market,0
1,10120,Yannawa,13.708,100.456,Convenience Store,Noodle House,Thai Restaurant,Coffee Shop,Som Tum Restaurant,Seafood Restaurant,Train Station,Gym / Fitness Center,Fast Food Restaurant,Market,0
2,10170,Thawi Wattana,13.767,100.450,Noodle House,Coffee Shop,Thai Restaurant,Café,Japanese Restaurant,Asian Restaurant,Fast Food Restaurant,Restaurant,Dessert Shop,Market,1
3,10100,Pom Prap Sattru Phai,13.733,100.500,Coffee Shop,Noodle House,Dessert Shop,Chinese Restaurant,Seafood Restaurant,Thai Restaurant,Hotpot Restaurant,Hotel,Shopping Mall,Café,2
4,10100,Samphanthawong,13.731,100.514,Noodle House,Hotel,Chinese Restaurant,Coffee Shop,Thai Restaurant,Spa,Bar,Dessert Shop,Hotel Bar,Art Gallery,2


In [41]:
latitude = 13.782733
longitude = 100.544291
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(bangkok_final['Latitude'], bangkok_final['Longitude'], bangkok_final['Neighborhood'], bangkok_final['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=7,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.9).add_to(map_clusters)
       
map_clusters

<img src="https://3.bp.blogspot.com/-dk9a-2JcjDQ/VKGwb3gSRbI/AAAAAAAACPI/cTc5Gea63eE/s1600/bangkok_density.png">

From the cluster details, we found that most neighborhoods in the area fall into the first group (0th). </br>

Let's explore them in more details

### Group 0 ###
Developing area

In [42]:
bangkok_final[bangkok_final['Cluster Labels'] == 0].head(10)

,PostalCode,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
0,10110,Wattana,13.708,100.456,Convenience Store,Noodle House,Thai Restaurant,Coffee Shop,Som Tum Restaurant,Seafood Restaurant,Train Station,Gym / Fitness Center,Fast Food Restaurant,Market,0
1,10120,Yannawa,13.708,100.456,Convenience Store,Noodle House,Thai Restaurant,Coffee Shop,Som Tum Restaurant,Seafood Restaurant,Train Station,Gym / Fitness Center,Fast Food Restaurant,Market,0
5,10120,Bang Kholame,13.708,100.456,Convenience Store,Noodle House,Thai Restaurant,Coffee Shop,Som Tum Restaurant,Seafood Restaurant,Train Station,Gym / Fitness Center,Fast Food Restaurant,Market,0
8,10150,Bang Bon,13.700,100.467,Convenience Store,Noodle House,Coffee Shop,Thai Restaurant,Chinese Restaurant,Seafood Restaurant,Steakhouse,Supermarket,Ice Cream Shop,BBQ Joint,0
22,10250,Pra Vet,13.858,100.469,Convenience Store,Thai Restaurant,Som Tum Restaurant,Seafood Restaurant,Noodle House,Restaurant,Market,Food Truck,Resort,Steakhouse,0
23,10250,Suang Luang,13.858,100.469,Convenience Store,Thai Restaurant,Som Tum Restaurant,Seafood Restaurant,Noodle House,Restaurant,Market,Food Truck,Resort,Steakhouse,0
34,10530,Nong Chok,13.856,100.862,Convenience Store,Dessert Shop,Shopping Mall,Thai Restaurant,Fish & Chips Shop,Food Truck,Construction & Landscaping,Pub,Coffee Shop,Restaurant,0
35,10600,Bangkok Yai,13.723,100.476,Noodle House,Convenience Store,Asian Restaurant,Train Station,Thai Restaurant,Seafood Restaurant,Dessert Shop,Steakhouse,Japanese Restaurant,Coffee Shop,0


### Group 1 ###
Upper-Middle class local people - Noodle House, Chinese and Thai restaurant

In [43]:
bangkok_final[bangkok_final['Cluster Labels'] == 1].head(10)

,PostalCode,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
2,10170,Thawi Wattana,13.767000,100.450000,Noodle House,Coffee Shop,Thai Restaurant,Café,Japanese Restaurant,Asian Restaurant,Fast Food Restaurant,Restaurant,Dessert Shop,Market,1
13,10170,Taling Chan,13.777000,100.457000,Noodle House,Café,Coffee Shop,Thai Restaurant,Japanese Restaurant,Asian Restaurant,Dessert Shop,Supermarket,Bakery,Sushi Restaurant,1
19,10230,Lat Phrao,13.825544,100.604447,Noodle House,Coffee Shop,Chinese Restaurant,Café,Vietnamese Restaurant,BBQ Joint,Thai Restaurant,Japanese Restaurant,Seafood Restaurant,Som Tum Restaurant,1
25,10300,Dusit,13.777000,100.520000,Noodle House,Thai Restaurant,Coffee Shop,Chinese Restaurant,Café,Palace,Hotel,Restaurant,Asian Restaurant,Museum,1
26,10310,Wangthong Lang,13.783000,100.517000,Noodle House,Thai Restaurant,Coffee Shop,Palace,Café,Hotel,Asian Restaurant,Chinese Restaurant,Museum,Japanese Restaurant,1
27,10320,Huai Khwang,13.783000,100.517000,Noodle House,Thai Restaurant,Coffee Shop,Palace,Café,Hotel,Asian Restaurant,Chinese Restaurant,Museum,Japanese Restaurant,1
28,10330,Pahtum Wan,13.783000,100.517000,Noodle House,Thai Restaurant,Coffee Shop,Palace,Café,Hotel,Asian Restaurant,Chinese Restaurant,Museum,Japanese Restaurant,1
37,10600,Thon Buri,13.725000,100.485000,Noodle House,Seafood Restaurant,Thai Restaurant,Asian Restaurant,Coffee Shop,Steakhouse,Dessert Shop,Train Station,Japanese Restaurant,BBQ Joint,1


### Group 2 ###
Fancy area for tourism - Bar, Spa, Hotel

In [44]:
bangkok_final[bangkok_final['Cluster Labels'] == 2].head(10)

,PostalCode,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
3,10100,Pom Prap Sattru Phai,13.733,100.500,Coffee Shop,Noodle House,Dessert Shop,Chinese Restaurant,Seafood Restaurant,Thai Restaurant,Hotpot Restaurant,Hotel,Shopping Mall,Café,2
4,10100,Samphanthawong,13.731,100.514,Noodle House,Hotel,Chinese Restaurant,Coffee Shop,Thai Restaurant,Spa,Bar,Dessert Shop,Hotel Bar,Art Gallery,2
14,10200,Phra Nakhon,13.754,100.501,Bar,Thai Restaurant,Café,Asian Restaurant,Massage Studio,Noodle House,Coffee Shop,Palace,Hostel,Bakery,2
29,10400,Ding Daeng,13.763,100.531,Hotel,Coffee Shop,Hostel,Steakhouse,Noodle House,Bar,Massage Studio,Café,Thai Restaurant,Japanese Restaurant,2
30,10500,Bang Rak,13.731,100.524,Hotel,Noodle House,Café,Thai Restaurant,Spa,Chinese Restaurant,French Restaurant,Coffee Shop,Seafood Restaurant,Restaurant,2
36,10600,Khlong San,13.731,100.509,Chinese Restaurant,Noodle House,Coffee Shop,Dessert Shop,Bar,Hotel Bar,Hotel,Hostel,Art Gallery,Spa,2
38,10700,Bang Plat,13.750,100.483,Café,Noodle House,Dessert Shop,Coffee Shop,Thai Restaurant,Palace,Bar,History Museum,Park,Pharmacy,2
39,10700,Bangkok Noi,13.763,100.478,Noodle House,Dessert Shop,Café,Chinese Restaurant,Coffee Shop,Bakery,History Museum,Hotpot Restaurant,Thai Restaurant,Park,2
43,10400,Phaya Thai,13.763,100.531,Hotel,Coffee Shop,Hostel,Steakhouse,Noodle House,Bar,Massage Studio,Café,Thai Restaurant,Japanese Restaurant,2


### Group 3 ###
Middle class area for local people

In [45]:
bangkok_final[bangkok_final['Cluster Labels'] == 3].head(10)

,PostalCode,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
6,10140,Thung Khru,13.683000,100.500000,Thai Restaurant,Coffee Shop,Noodle House,Convenience Store,Hotpot Restaurant,Chinese Restaurant,BBQ Joint,Seafood Restaurant,Café,Market,3
7,10140,Rat Burana,13.682000,100.506000,Coffee Shop,Thai Restaurant,Noodle House,Hotpot Restaurant,Chinese Restaurant,Pub,Convenience Store,Seafood Restaurant,Market,BBQ Joint,3
9,10150,Bang Khun Thian,13.663000,100.434000,Thai Restaurant,Coffee Shop,Noodle House,Ice Cream Shop,Hotpot Restaurant,Japanese Restaurant,Seafood Restaurant,Restaurant,Bakery,Department Store,3
10,10160,Nong Khaem,13.705000,100.349000,Coffee Shop,Thai Restaurant,Hotpot Restaurant,Restaurant,Market,Flea Market,Shopping Mall,Big Box Store,Ice Cream Shop,Som Tum Restaurant,3
11,10160,Phasi Charoen,13.715000,100.437000,Coffee Shop,Convenience Store,BBQ Joint,Thai Restaurant,Noodle House,Japanese Restaurant,Fast Food Restaurant,Hotpot Restaurant,Department Store,Shopping Mall,3
12,10160,Bang Khae,13.719000,100.393000,Noodle House,Coffee Shop,Thai Restaurant,Café,BBQ Joint,Dessert Shop,Japanese Restaurant,Ice Cream Shop,Hotpot Restaurant,Convenience Store,3
15,10210,Lak Si,13.877388,100.569273,Coffee Shop,Thai Restaurant,Hotel,Convenience Store,Noodle House,Gym / Fitness Center,Bar,Chinese Restaurant,Japanese Restaurant,Som Tum Restaurant,3
16,10220,Sai Mai,13.867000,100.600000,Noodle House,Coffee Shop,Thai Restaurant,Asian Restaurant,Bookstore,Vietnamese Restaurant,Convenience Store,Fast Food Restaurant,Hotpot Restaurant,Shabu-Shabu Restaurant,3
17,10220,Bang Khen,13.877000,100.625000,Thai Restaurant,Noodle House,Coffee Shop,Convenience Store,Som Tum Restaurant,Seafood Restaurant,Asian Restaurant,Bakery,Steakhouse,Fast Food Restaurant,3
18,10230,Khanna Yao,13.819628,100.679240,Thai Restaurant,Japanese Restaurant,Coffee Shop,Café,Asian Restaurant,Seafood Restaurant,Bakery,Dessert Shop,Shopping Mall,Restaurant,3
